In [1]:
def recombination(i):
    new_lst = []
    for block in range(len(i)):
        if 0 not in i[block]:
            # Remove block
            lst = i[:block]+i[block+1:]
            # Split block
            lst = lst + [(i[block][0], 0), (0, i[block][1])]
            # Append to list
            new_lst.append(sorted(lst))
    return sorted(new_lst)

i = [(2, 2), (3, 3), (4, 4)]
recombination(i)

[[(0, 2), (2, 0), (3, 3), (4, 4)],
 [(0, 3), (2, 2), (3, 0), (4, 4)],
 [(0, 4), (2, 2), (3, 3), (4, 0)]]

In [2]:
def coalescent(i):
    new_lst = []
    # For each combination of blocks
    for b1 in range(len(i)):
        for b2 in range(b1+1, len(i)):
            # Create new block by merging
            add_lst = [(i[b1][0]+i[b2][0], i[b1][1]+i[b2][1])]
            for b3 in range(len(i)):
                if b3 not in [b1, b2]:
                    add_lst.append(i[b3])
            new_lst.append(sorted(add_lst))
    return sorted(new_lst)
i = [(0, 2), (0, 3), (0, 4), (2, 0), (3, 0), (4, 0)]
coalescent(i)

[[(0, 2), (0, 3), (0, 4), (2, 0), (7, 0)],
 [(0, 2), (0, 3), (0, 4), (3, 0), (6, 0)],
 [(0, 2), (0, 3), (0, 4), (4, 0), (5, 0)],
 [(0, 2), (0, 3), (2, 0), (3, 0), (4, 4)],
 [(0, 2), (0, 3), (2, 0), (3, 4), (4, 0)],
 [(0, 2), (0, 3), (2, 4), (3, 0), (4, 0)],
 [(0, 2), (0, 4), (2, 0), (3, 0), (4, 3)],
 [(0, 2), (0, 4), (2, 0), (3, 3), (4, 0)],
 [(0, 2), (0, 4), (2, 3), (3, 0), (4, 0)],
 [(0, 2), (0, 7), (2, 0), (3, 0), (4, 0)],
 [(0, 3), (0, 4), (2, 0), (3, 0), (4, 2)],
 [(0, 3), (0, 4), (2, 0), (3, 2), (4, 0)],
 [(0, 3), (0, 4), (2, 2), (3, 0), (4, 0)],
 [(0, 3), (0, 6), (2, 0), (3, 0), (4, 0)],
 [(0, 4), (0, 5), (2, 0), (3, 0), (4, 0)]]

In [3]:
def get_states(lst):
    iter_lst_0 = [sorted(i) for i in lst]
    one_iteration = iter_lst_0
    n0 = 0
    n1 = len(iter_lst_0)
    while n0 < n1:
        for i in one_iteration:
            iter_lst_0 = iter_lst_0+recombination(i)
            iter_lst_0 = iter_lst_0+coalescent(i)
        iter_lst_0 = sorted(list(list(i) for i in set(tuple(row) for row in iter_lst_0)))
        n0 = len(one_iteration)
        n1 = len(iter_lst_0)
        one_iteration = iter_lst_0
    return sorted(iter_lst_0)

In [4]:
state_2 = [[(2, 2)]]
iter_lst_0 = get_states(state_2)
iter_lst_0

[[(0, 2), (2, 0)], [(2, 2)]]

In [5]:
state_4 = [
    [(2, 2), (3, 3)]]
iter_lst = get_states(state_4)
iter_lst

[[(0, 2), (0, 3), (2, 0), (3, 0)],
 [(0, 2), (0, 3), (5, 0)],
 [(0, 2), (2, 0), (3, 3)],
 [(0, 2), (2, 3), (3, 0)],
 [(0, 2), (5, 3)],
 [(0, 3), (2, 0), (3, 2)],
 [(0, 3), (2, 2), (3, 0)],
 [(0, 3), (5, 2)],
 [(0, 5), (2, 0), (3, 0)],
 [(0, 5), (5, 0)],
 [(2, 0), (3, 5)],
 [(2, 2), (3, 3)],
 [(2, 3), (3, 2)],
 [(2, 5), (3, 0)],
 [(5, 5)]]

In [6]:
state_15 = iter_lst

state_15 = [sorted(i) for i in state_15]

print(len(state_15))

for i in state_15:
    l = 0
    r = 0
    for j in i:
        l += j[0]
        r += j[1]
    print((l, r))

15
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 5)


In [7]:
new_state = [[(4, 4)], [(4, 0), (0, 4)]]
starting_states = []

for i in new_state:
    for j in state_15:
        starting_states.append(sorted(i+j))
        
starting_states.sort()

print(len(starting_states))

starting_states

30


[[(0, 2), (0, 3), (0, 4), (2, 0), (3, 0), (4, 0)],
 [(0, 2), (0, 3), (0, 4), (4, 0), (5, 0)],
 [(0, 2), (0, 3), (2, 0), (3, 0), (4, 4)],
 [(0, 2), (0, 3), (4, 4), (5, 0)],
 [(0, 2), (0, 4), (2, 0), (3, 3), (4, 0)],
 [(0, 2), (0, 4), (2, 3), (3, 0), (4, 0)],
 [(0, 2), (0, 4), (4, 0), (5, 3)],
 [(0, 2), (2, 0), (3, 3), (4, 4)],
 [(0, 2), (2, 3), (3, 0), (4, 4)],
 [(0, 2), (4, 4), (5, 3)],
 [(0, 3), (0, 4), (2, 0), (3, 2), (4, 0)],
 [(0, 3), (0, 4), (2, 2), (3, 0), (4, 0)],
 [(0, 3), (0, 4), (4, 0), (5, 2)],
 [(0, 3), (2, 0), (3, 2), (4, 4)],
 [(0, 3), (2, 2), (3, 0), (4, 4)],
 [(0, 3), (4, 4), (5, 2)],
 [(0, 4), (0, 5), (2, 0), (3, 0), (4, 0)],
 [(0, 4), (0, 5), (4, 0), (5, 0)],
 [(0, 4), (2, 0), (3, 5), (4, 0)],
 [(0, 4), (2, 2), (3, 3), (4, 0)],
 [(0, 4), (2, 3), (3, 2), (4, 0)],
 [(0, 4), (2, 5), (3, 0), (4, 0)],
 [(0, 4), (4, 0), (5, 5)],
 [(0, 5), (2, 0), (3, 0), (4, 4)],
 [(0, 5), (4, 4), (5, 0)],
 [(2, 0), (3, 5), (4, 4)],
 [(2, 2), (3, 3), (4, 4)],
 [(2, 3), (3, 2), (4, 4)],
 [(2

In [8]:
for i in starting_states:
    l = 0
    r = 0
    for j in i:
        l += j[0]
        r += j[1]
    print((l, r))

(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)
(9, 9)


In [8]:
state_n = [[(2, 2), (3, 3), (4, 4)]]
iter_lst_2 = get_states(state_n)
len(iter_lst_2)

203

In [9]:
state_n = [[(2, 2), (3, 3), (4, 4)]]
len(get_states(state_n))

203

In [14]:
def colored(r, g, b, text):
    return "\033[38;2;{};{};{}m{}\033[38;2;000;000;000m".format(r, g, b, text)
dct = {
    2:('204', '000', '000'),
    3:('000', '000', '204'),
    4:('204', '204', '000'),
    5:('204', '000', '204'),
    6:('204', '102', '000'),
    7:('000', '204', '000'),
    9:('000', '000', '000'),
}

for i in dct:
    entry = dct[i]
    print('-'+colored(entry[0], entry[1], entry[2], i))

-2
-3
-4
-5
-6
-7
-9


In [15]:

def print_states(i, fill):
    string = ' ______ \n'
    for j in i:
        string += '| '
        if j[0] == 0:
            string += '  '
        else:
            entry = dct[j[0]]
            if j[0] < 5:
                symbol = '●'
            else:
                symbol = 'X'
            string += (colored(entry[0], entry[1], entry[2], symbol)+'-')
        if j[1] == 0:
            string += '  '
        else:
            if j[1] < 5:
                symbol = '●'
            else:
                symbol = 'X'
            entry = dct[j[1]]
            string += ('-'+colored(entry[0], entry[1], entry[2], symbol))
        string += ' |\n'
    string += ' ‾‾‾‾‾‾ '
    string += '\n        '*(fill-len(string.split('\n')))
    return string

print(print_states([(0, 2), (0, 3), (0, 4), (2, 0), (7, 0)], 7))

 ______ 
|   -● |
|   -● |
|   -● |
| ●-   |
| X-   |
 ‾‾‾‾‾‾ 


In [12]:


def print_all_states(iter_lst, n_col):
    max_len = max([len(i) for i in iter_lst])+2
    x = []
    for i in iter_lst:
        x.append(print_states(i, max_len))
    new_string = []
    for row in range(0, len(x), n_col):
        end = row+n_col
        if end > len(x):
            end = len(x)
        string = ''
        for max_depth in range(max_len):
            for i in range(row, end):
                if len(x[i].split('\n')) > max_depth:
                    string += x[i].split('\n')[max_depth]
            string += '\n'
        new_string.append(string)
    
    return new_string

def print_all_states(iter_lst, n_col):
    
    new_string = []
    for row in range(0, len(iter_lst), n_col):
        
        end = row+n_col
        if end > len(iter_lst):
            end = len(iter_lst)
        
        max_len = max([len(i) for i in iter_lst[row:end]])+2
        x = []
        for i in iter_lst[row:end]:
            x.append(print_states(i, max_len))
                    
        string = ''
        for max_depth in range(max_len):
            for i in x:
                string += i.split('\n')[max_depth]
            string += '\n'
        new_string.append(string[:-2])
    
    return new_string




In [13]:
for i in print_all_states(iter_lst_0, 1):
    print(i)

 ______ 
|   -● |
| ●-   |
 ‾‾‾‾‾‾
 ______ 
| ●--● |
 ‾‾‾‾‾‾


In [14]:
for i in print_all_states(iter_lst, 5):
    print(i)

 ______  ______  ______  ______  ______ 
|   -● ||   -● ||   -● ||   -● ||   -● |
|   -● ||   -● || ●-   || ●--● || X--● |
| ●-   || X-   || ●--● || ●-   | ‾‾‾‾‾‾ 
| ●-   | ‾‾‾‾‾‾  ‾‾‾‾‾‾  ‾‾‾‾‾‾         
 ‾‾‾‾‾‾                                
 ______  ______  ______  ______  ______ 
|   -● ||   -● ||   -● ||   -X ||   -X |
| ●-   || ●--● || X--● || ●-   || X-   |
| ●--● || ●-   | ‾‾‾‾‾‾ | ●-   | ‾‾‾‾‾‾ 
 ‾‾‾‾‾‾  ‾‾‾‾‾‾          ‾‾‾‾‾‾        
 ______  ______  ______  ______  ______ 
| ●-   || ●--● || ●--● || ●--X || X--X |
| ●--X || ●--● || ●--● || ●-   | ‾‾‾‾‾‾ 
 ‾‾‾‾‾‾  ‾‾‾‾‾‾  ‾‾‾‾‾‾  ‾‾‾‾‾‾        


In [15]:
for i in print_all_states(iter_lst_2, 10):
    print(i)

 ______  ______  ______  ______  ______  ______  ______  ______  ______  ______ 
|   -● ||   -● ||   -● ||   -● ||   -● ||   -● ||   -● ||   -● ||   -● ||   -● |
|   -● ||   -● ||   -● ||   -● ||   -● ||   -● ||   -● ||   -● ||   -● ||   -● |
|   -● ||   -● ||   -● ||   -● ||   -● || ●-   || ●-   || ●-   || ●--● || ●--● |
| ●-   || ●-   || ●-   || ●-   || X-   || ●-   || ●--● || X--● || ●-   || X-   |
| ●-   || X-   || X-   || X-   | ‾‾‾‾‾‾ | ●--● || ●-   | ‾‾‾‾‾‾ | ●-   | ‾‾‾‾‾‾ 
| ●-   | ‾‾‾‾‾‾  ‾‾‾‾‾‾  ‾‾‾‾‾‾          ‾‾‾‾‾‾  ‾‾‾‾‾‾          ‾‾‾‾‾‾         
 ‾‾‾‾‾‾                                                                        
 ______  ______  ______  ______  ______  ______  ______  ______  ______  ______ 
|   -● ||   -● ||   -● ||   -● ||   -● ||   -● ||   -● ||   -● ||   -● ||   -● |
|   -● ||   -● ||   -● ||   -● ||   -● ||   -● ||   -● ||   -● ||   -● ||   -● |
| ●-   || ●--● || ●-   || ●--● || X--● || ●-   || ●-   || ●-   || ●--● || ●--● |
| X--● || X-   || X--● || X- 